<a href="https://colab.research.google.com/github/nedokormysh/GB_NLP_intro/blob/course_work/GB_NLP_course_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Курсовой проект «Введение в обработку естественного языка»
**Задание:**

Реализовать чат-бот на базе API Telegram.

**Интенты:**

* болталка (разговорная часть): используются вопросы-ответы mail.ru,
* суммаризация: используется предобученная модель - csebuetnlp/mT5_multilingual_XLSum
* товарный запрос: используются данные с youla.ru.
* прогноз погоды

Технологии:

ML: CountVectorizer, TfidfVectorizer, FastText, MorphAnalyzer, dialogflow, LogisticRegression, annoy, API: telegram

**Алгоритм работы чат-бота:**

При поступлении текстового запроса модель должна определить интент к которому относится запрос:

болталка (вопрос/ответ)
суммаризация
поиск продукта


В разговорном режиме, используя TfidfVectorizer, FastText, annoy определяется наиболее подходящий ответ.

Если интент "cуммаризация", то используется предобученная модель csebuetnlp/mT5_multilingual_XLSum. Суммаризация должна начинаться с ключевого слова "Summarization:".

Если запрос товарный, используя TfidfVectorizer, FastText, annoy (быстрый поиск ближайших соседей). Определяются 3 наиболее похожих продукта, которые возвращяются в чат.

Если в разговорном и продуктовом запросе найденный ответ слабо соответствует запросу, чат-бот должен ответить "Не понимаю запрос. Сформулируйте запрос ещё раз.".``

В прогнозе погоды выводим погоду в городе. Если не может понять, то выводится сообщение, что не может определить погодные условия.

# 1. Установка библиотек и подготовка данных.

In [ ]:
!pip install telegram --q
!pip install python-telegram-bot --q
!pip install pymorphy2 --q
!pip install stop_words --q
!pip install annoy --q
!pip install transformers sentencepiece --q

!pip3 uninstall python-telegram-bot -q

# !pip3 install python-telegram-bot
!pip install python-telegram-bot==13.13

Proceed (Y/n)? н
Your response ('н') was not one of the expected responses: y, n, 
Proceed (Y/n)? y
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached python_telegram_bot-13.13-py3-none-any.whl (513 kB)


In [ ]:
import os
from telegram.ext import Updater, CommandHandler, MessageHandler, filters, CallbackContext
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Обучение языковой модели

In [ ]:
%%time

assert True

#Small preprocess of the answers

question = None
written = False

c=0
i=0
# Идем по всем записям, берем строку как вопрос и после знака "---" находим ответ
with open("prepared_answers.txt", "w") as fout:
    with open("/content/drive/MyDrive/GB_NLP_intro_course_work/train_.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue
            i+=1

<timed exec>:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

CPU times: user 1.27 s, sys: 209 ms, total: 1.48 s
Wall time: 1.79 s


## Предобработка текста.

In [ ]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [ ]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

Изначально файл train_ был получен из датасета https://www.kaggle.com/datasets/atleast6characterss/otvetmailru-solved-questions . Датасет большой по объёму, поэтому была вырезана часть текста.

In [ ]:
assert True

# Preprocess for models fitting

sentences = []
c = 0

with open("/content/drive/MyDrive/GB_NLP_intro_course_work/train_.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

<ipython-input-7-d1511a9a2194>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

In [ ]:
sentences = [i for i in sentences if len(i) > 2]

# записываем сериализованный объект в файл
with open(f'sentences.pkl', 'wb') as f:
    pickle.dump(sentences, f)

In [ ]:
# загружаем объект из файла
with open(f'sentences.pkl', 'rb') as f:
    sentences = pickle.load(f)

In [ ]:
sentences[0]

['отдача', 'замучить', 'ущипнуть']

## Обучение tf-idf

In [ ]:
# Функция-пустышка, для возможности передать в TfidfVectorizer список уже готовых токенов, а не текст
def simple_tokenizer(x):
    return x

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
%%time
tfidf_vectorizer = TfidfVectorizer(tokenizer=simple_tokenizer, lowercase=False, min_df=2)
tfidf_vectorizer.fit_transform(sentences)

idfs = {v[0]: v[1] for v in zip(tfidf_vectorizer.vocabulary_, tfidf_vectorizer.idf_)}
midf = np.mean(tfidf_vectorizer.idf_)

with open(f'idfs.pkl', 'wb') as f:
    pickle.dump(idfs, f)  # записывает сериализованный объект в файл.
    
with open(f'midf.pkl', 'wb') as f:
    pickle.dump(midf, f)  # записывает сериализованный объект в файл.

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CPU times: user 3.78 s, sys: 111 ms, total: 3.89 s
Wall time: 3.94 s


In [ ]:
with open(f'idfs.pkl', 'rb') as f:
    idfs = pickle.load(f)  # загружает объект из файла

In [ ]:
with open(f'midf.pkl', 'rb') as f:
    midf = pickle.load(f)  # загружает объект из файла

## Обучение модели FastText

In [ ]:
SIZE_EMB = 200  # Размер эмбеддинга

In [ ]:
def embed_txt(txt, idfs, model, midf):
    n_ft = 0
    vector_ft = np.zeros(SIZE_EMB)
    for word in txt:
        if word in model:
            vector_ft += model[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
        
    return vector_ft

In [ ]:
%%time
modelFT = FastText(sentences=sentences, vector_size=SIZE_EMB, min_count=2, window=5, workers=8, seed=34)
modelFT.save("ft_model")

CPU times: user 6min 43s, sys: 7.27 s, total: 6min 51s
Wall time: 4min 29s


In [ ]:
%%time

modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(SIZE_EMB ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = re.sub(r'\<[^>]*\>', '', spls[1]) # Удалим html-тэги
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(SIZE_EMB)

        for word in question:
            # print(word)
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                # n_ft += 1
                n_ft += idfs.get(word, midf)

        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        if counter > 1000000:
            break

ft_index.build(10)
ft_index.save('ft_index.ann')

# Сохраняем индекс вопросов из разговорной части
with open(f'index_map.pkl', 'wb') as f:   # Save it for future use
    pickle.dump(index_map, f)  # записывает сериализованный объект в файл

<timed exec>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

CPU times: user 11min 1s, sys: 10.2 s, total: 11min 11s
Wall time: 11min 25s


In [ ]:
ft_index = annoy.AnnoyIndex(SIZE_EMB, 'angular')  # Создание Annoy индекса
ft_index.load(f'ft_index.ann')  # загружает 

True

In [ ]:
with open(f'index_map.pkl', 'rb') as f:
    index_map = pickle.load(f)  # загружает объект из файла

In [ ]:
ft_index.get_nns_by_vector(np.zeros(SIZE_EMB), 2)

[396, 1518]

# Обучение продуктовых моделей

In [ ]:
%%time

shop_data = pd.read_csv("/content/drive/MyDrive/GB_NLP_intro_course_work/ProductsDataset.csv")
shop_data['text'] = shop_data['title'] + " " + shop_data["descrirption"]
shop_data['text'] = shop_data['text'].apply(lambda x: preprocess_txt(str(x)))
shop_data.head()

CPU times: user 2min 55s, sys: 501 ms, total: 2min 55s
Wall time: 2min 57s


,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, новый, носить, реал, кра..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))

In [ ]:
%%time

idxs = set(np.random.randint(0, len(index_map), len(shop_data)))
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in idxs]
positive_texts = [" ".join(val) for val in shop_data['text'].values]

CPU times: user 4min 2s, sys: 577 ms, total: 4min 2s
Wall time: 4min 5s


In [ ]:
dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, stratify=labels,
                                                    random_state=7575)

Сконвертируем набор текстов в токены при помощи CountVectorizer 

In [ ]:
%%time

x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

CPU times: user 21 s, sys: 10.2 s, total: 31.3 s
Wall time: 24.4 s


In [ ]:
with open(f'vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1, 2))
with open(f'vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

In [ ]:
with open(f'lr.pkl', 'wb') as f:
    pickle.dump(lr, f)

In [ ]:
lr = LogisticRegression()
with open(f'lr.pkl', 'rb') as f:
    lr = pickle.load(f)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.9819663472421338

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(X_train)

## Обучение модели TF-IDF

In [ ]:
%%time
tfidf_vect_prod = TfidfVectorizer(lowercase=False, min_df=2)
tfidf_vect_prod.fit(X_train)

idfs_prod = {v[0]: v[1] for v in zip(tfidf_vect_prod.vocabulary_, tfidf_vect_prod.idf_)}
midf_prod = np.mean(tfidf_vect_prod.idf_)

with open(f'idfs_prod.pkl', 'wb') as f:
    pickle.dump(idfs_prod, f)
with open(f'midf_prod.pkl', 'wb') as f:
    pickle.dump(midf_prod, f)

CPU times: user 1.23 s, sys: 15 ms, total: 1.25 s
Wall time: 1.26 s


In [ ]:
with open(f'idfs_prod.pkl', 'rb') as f:
    idfs_prod = pickle.load(f)

In [ ]:
with open(f'midf_prod.pkl', 'rb') as f:
    midf_prod = pickle.load(f)

## Annoy. Алгоритм приблизительного поиска

In [ ]:
%%time
ft_index_shop = annoy.AnnoyIndex(SIZE_EMB ,'angular')
index_map_shop = {}
counter = 0

for i in tqdm_notebook(range(len(shop_data))):
    n_ft = 0
    index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])
    vector_ft = np.zeros(SIZE_EMB)
    for word in shop_data.loc[i, "text"]:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * idfs.get(word, midf_prod)
            n_ft += idfs.get(word, midf_prod)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_shop.add_item(counter, vector_ft)
    counter += 1

ft_index_shop.build(50)
# ft_index_shop.save('shop.ann')
ft_index_shop.save(f'ft_index_shop')

with open(f'index_map_shop.pkl', 'wb') as f:
    pickle.dump(index_map_shop, f)  # записывает сериализованный объект в файл.

<timed exec>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/35548 [00:00<?, ?it/s]

CPU times: user 22.6 s, sys: 1.47 s, total: 24 s
Wall time: 21.6 s


In [ ]:
ft_index_shop = annoy.AnnoyIndex(SIZE_EMB, 'angular')
ft_index_shop.load(f'ft_index_shop') 

True

In [ ]:
with open(f'index_map_shop.pkl', 'rb') as f:
    index_map_shop = pickle.load(f)

# Погода

In [43]:
# Список ключевых слов для определения запроса о погоде

list_for_weather = ['погода','солнечно', 'осадка', 'дождь', 'снег', 'ливень', 'зонт']

In [44]:
# Проверка есть ли совпадение со списком ключевых слов

def is_in_list(text, list):
    for word in text:
        if word in list:
            return True
            break
    return False

In [45]:
# Запрос прогноза погоды

import requests 
import json

key = '18d7be471bb6490092f155952231804' # Токен https://www.weatherapi.com/

def get_weather(city):
    url = f'http://api.weatherapi.com/v1/current.json?key={key}&q={city}&lang=ru'
    requests.get(url)
    response = requests.get(url)
    json_data = json.loads(response.text)

    city = json_data['location']['name']
    time = json_data['location']['localtime'].split(' ')[1]
    temp = json_data['current']['temp_c']
    temp_fl = json_data['current']['feelslike_c']
    condition = json_data['current']['condition']['text']
    wind = json_data['current']['wind_mph']
    uv = json_data['current']['uv']

    msg = f'В городе {city} сейчас {time}. О погоде: {condition}, температура {temp} градусов, ощущается как {temp_fl} градусов, скорость ветра {wind} м/ч, уровень ультрафиолета {uv}'

    return msg

In [46]:
# Определение города

import spacy

nlp = spacy.load("ru_core_news_sm")

def location_recognition(text): 
    locs = []
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == 'LOC':
            locs.append(ent.text)
    locs = [morpher.parse(i.lower())[0].normal_form for i in locs]
    return locs

/usr/local/lib/python3.9/dist-packages/spacy/util.py:887: UserWarning: [W095] Model 'ru_core_news_sm' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.5.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [59]:
# Погода в заданном городе

def forecast(text):
    city = location_recognition(text)
    try:
        forecast = get_weather(city)
    except:
        forecast = 'Не могу определить погодные условия.'
    
    return forecast

# Бот

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import MBartTokenizer, MBartForConditionalGeneration

In [ ]:
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_mT5_multilingual_XLSum = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [41]:
import os
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd
import re

from telegram import Update
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

SIZE_EMB = 200  # Размер эмбеддинга

In [ ]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [ ]:
with open(f'idfs.pkl', 'rb') as f:
    idfs = pickle.load(f)

with open(f'midf.pkl', 'rb') as f:
    midf = pickle.load(f)

with open(f'idfs_prod.pkl', 'rb') as f:
    idfs_prod = pickle.load(f)  # загружает объект из файла

with open(f'midf_prod.pkl', 'rb') as f:
    midf_prod = pickle.load(f)  # загружает объект из файла

# Подгружаем предварительно обученную модель вопросов из болталки
modelFT = FastText.load(f'ft_model')

# Подгружаем предварительно подготовленный и сохранненый индекс ответов из болталки
ft_index = annoy.AnnoyIndex(SIZE_EMB, 'angular')
ft_index.load(f'/content/ft_index.ann')
with open(f'/content/index_map.pkl', 'rb') as f:
    index_map = pickle.load(f)  # загружает объект из файла

vectorizer = CountVectorizer(ngram_range=(1, 2))
with open(f'vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

lr = LogisticRegression()
with open(f'lr.pkl', 'rb') as f:
    lr = pickle.load(f)

with open(f'midf_prod.pkl', 'rb') as f:
    midf_p = pickle.load(f)

ft_index_shop = annoy.AnnoyIndex(SIZE_EMB, 'angular')
ft_index_shop.load(f'ft_index_shop') 

# Подгружаем предварительно обученную модель продуктовых названий
with open(f'index_map_shop.pkl', 'rb') as f:
    index_map_shop = pickle.load(f)

## Работа бота

In [ ]:
def echo(update: Update, context: CallbackContext):
    txt = update.message.text
    update.message.reply_text('Ваше сообщение! ' + update.message.text)


def startCommand(update: Update, context: CallbackContext) -> None:
    update.message.reply_text('Добрый день!')


In [ ]:
# !pip install python-telegram-bot==13.13

In [61]:
updater = Updater("5928534129:AAGIomj6LYU8ljifCbL_cVkfP2Q4VuTjrbA"
# , use_context=True
) # Токен API к Telegram

def echo(update: Update, context: CallbackContext):
    txt = update.message.text
    update.message.reply_text('Ваше сообщение! ' + update.message.text)


def startCommand(update: Update, context: CallbackContext) -> None:
    update.message.reply_text('Добрый день!')


def model_mT5_multilingual_XLSum_summary(input_text, model, tokenizer): 
    """ 
        Суммаризация. Предобученную модель: csebuetnlp/mT5_multilingual_XLSum
    """
    WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

    input_ids = tokenizer(
        [WHITESPACE_HANDLER(input_text)],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        max_length=84,
        no_repeat_ngram_size=2,
        num_beams=4
    )[0]

    summary = tokenizer.decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )

    return summary

# Определение и выполнение запроса пользователя

def textMessage(update: Update, context: CallbackContext) -> None:
    
    # 1. Суммаризация. Начало фразы с ключевого слова "Summarization:"
    input_text = update.message.text
    if input_text.split(' ', 1)[0] == 'Summarisation:':
      update.message.reply_text('Подождите, идет суммаризация текста ...')

      input_text = input_text.replace('Summarisation:', '')
      summary = model_mT5_multilingual_XLSum_summary(input_text, model_mT5_multilingual_XLSum, tokenizer)
      update.message.reply_text('Суммаризация: ' + summary)
  

    else:
      res_text = 'Не понимаю. Сформулируйте запрос ещё раз.'
      input_txt = preprocess_txt(update.message.text)
      vect = vectorizer.transform([" ".join(input_txt)])
      prediction = lr.predict(vect)

      # 4. Прогноз погоды    
      if is_in_list(input_txt, list_for_weather) == True:        
          update.message.reply_text(forecast(update.message.text))
      
      # 2. Определим является ли запрос продуктовым.
      elif prediction[0] == 1:
          # Если запрос продуктовый, то найдём 3 самых подходящих товара.
          # update.message.reply_text('Продуктовый ...')
          find = False
          vect_ft = embed_txt(input_txt, idfs_prod, modelFT.wv, midf_prod)
          ft_index_shop_val, distances_shop = ft_index_shop.get_nns_by_vector(vect_ft, 3, include_distances=True)

          for i, item in enumerate(ft_index_shop_val):
              if distances_shop[i] <= 0.5:          
                title, image = index_map_shop[item]
                print(title, image)
                update.message.reply_text("title: {} image: {}".format(title, image))
                find = True
          if find == False:
            update.message.reply_text(res_text)
      
      else:
          # 3. Если запрос разговорный, то найдём ответ.
          # update.message.reply_text('Разговорный ...')
          vect_ft = embed_txt(input_txt, idfs, modelFT.wv, midf)
          ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
          if distances[0] <= 0.9:
              update.message.reply_text(index_map[ft_index_val[0]])
          else:
              update.message.reply_text(res_text)

In [62]:
dispatcher = updater.dispatcher  # Диспетчер

# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler('start', startCommand))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, textMessage))

# Start Bot
updater.start_polling()
updater.idle()

Ботинки http://cache3.youla.io/files/images/360_360/59/8d/598d8651ec985587912f7804.jpg
Сапоги http://cache3.youla.io/files/images/360_360/58/cd/58cd50fa1c40313e10758adb.jpg
Коляска зима - лето http://cache3.youla.io/files/images/360_360/5b/fb/5bfbd2479380005ed0275fc2.jpg
